<a href="https://colab.research.google.com/github/guanyu1127/Programming-Language/blob/main/HW6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [13]:
# 這是 Colab 筆記本的第一個儲存格
# 我們只需要 Google Sheets 相關套件
!pip install -q gspread gspread-dataframe pandas

In [14]:
# 這是 Colab 筆記本的第二個儲存格
import pandas as pd
import gspread
import pytz
from gspread_dataframe import set_with_dataframe
from google.colab import auth
from google.auth import default
from datetime import datetime

# --- 授權 Google Sheets ---
print("正在授權 Google Sheets...")
auth.authenticate_user()
creds, _ = default()
gc = gspread.authorize(creds)
print("✅ Google Sheets 授權成功！")

正在授權 Google Sheets...
✅ Google Sheets 授權成功！


In [15]:
# 這是 Colab 筆記本的第三個儲存格

# ⬇️ ！！！請將 "..." 替換成你自己的 Google Sheet 網址 ！！！ ⬇️
SHEET_URL = "https://docs.google.com/spreadsheets/d/1pMxPxJPXe__0THBZlhqr6nmFSZzIOhHao2tl1JMXvg0/edit?usp=sharing"
# ⬆️ ！！！請將 "..." 替換成你自己的 Google Sheet 網址 ！！！ ⬆️

WORKSHEET_LOG = "Habit_Log"
WORKSHEET_SUMMARY = "Built_in_Summary" # 這是新的分頁名稱
TIMEZONE = "Asia/Taipei"

try:
    gsheets = gc.open_by_url(SHEET_URL)
    print(f"✅ 成功開啟 Google Sheet: '{gsheets.title}'")
except Exception as e:
    print(f"❌ 開啟 Google Sheet 失敗: {e}")

✅ 成功開啟 Google Sheet: 'hw6'


In [16]:
# 這是 Colab 筆記本的第四個儲存格
print(f"正在寫入範例資料至 '{WORKSHEET_LOG}' 分頁...")

example_data = [
    {"Date": "2025-11-09", "Habit_Name": "學日文", "Status": "✅", "Notes": "複習 N5 文法"},
    {"Date": "2025-11-09", "Habit_Name": "學 Python", "Status": "❌", "Notes": "太忙了"},
    {"Date": "2025-11-09", "Habit_Name": "運動", "Status": "✅", "Notes": "跑步 30 分鐘"},
    {"Date": "2025-11-10", "Habit_Name": "學日文", "Status": "✅", "Notes": "背 20 個單字"},
    {"Date": "2025-11-10", "Habit_Name": "學 Python", "Status": "✅", "Notes": "練習 for 迴圈"},
    {"Date": "2025-11-10", "Habit_Name": "運動", "Status": "❌", "Notes": "下雨"},
]
df_example = pd.DataFrame(example_data)

try:
    log_sheet = gsheets.worksheet(WORKSHEET_LOG)
    log_sheet.clear()
    set_with_dataframe(log_sheet, df_example)
    print(f"✅ 成功寫入 {len(df_example)} 筆範例資料！")
except Exception as e:
    print(f"❌ 寫入範例資料失敗: {e}")

正在寫入範例資料至 'Habit_Log' 分頁...
✅ 成功寫入 6 筆範例資料！


In [17]:
# 這是 Colab 筆記本的第五個儲存格
print(f"--- 正在執行【挑戰 1：資料擷取】---")

try:
    log_sheet = gsheets.worksheet(WORKSHEET_LOG)
    all_data = log_sheet.get_all_records()

    if not all_data:
        print(f"❌ 你的 '{WORKSHEET_LOG}' 分頁是空的。")
        # 建立一個空的 DataFrame 以避免後續錯誤
        df_log = pd.DataFrame(columns=["Date", "Habit_Name", "Status", "Notes"])
    else:
        df_log = pd.DataFrame(all_data)
        print(f"✅ 成功從 '{WORKSHEET_LOG}' 讀取 {len(df_log)} 筆資料。")
        display(df_log.tail()) # 顯示最後 5 筆資料

except Exception as e:
    print(f"❌ 讀取 '{WORKSHEET_LOG}' 失敗: {e}")

--- 正在執行【挑戰 1：資料擷取】---
✅ 成功從 'Habit_Log' 讀取 6 筆資料。


,Date,Habit_Name,Status,Notes
1,2025-11-09,學 Python,❌,太忙了
2,2025-11-09,運動,✅,跑步 30 分鐘
3,2025-11-10,學日文,✅,背 20 個單字
4,2025-11-10,學 Python,✅,練習 for 迴圈
5,2025-11-10,運動,❌,下雨


In [20]:
# 這是 Colab 筆記本的第六個儲存格 (全新加強版)
# 我們匯入 random 函式庫來產生隨機的鼓勵短語
import random

print(f"--- 正在執行【挑戰 2：內建規則總結 (加強版)】---")

def make_built_in_summary(df):
    """
    (加強版) 模仿老師範例 (hw6) 中的「方案 A」，
    不使用 AI，但使用更詳細的 Python 規則來生成「多行」總結句。
    """

    if df.empty:
        return "尚無紀錄可總結。試著新增一筆今天的習慣吧！"

    # --- 1. 準備隨機的鼓勵短語 ---
    good_job_phrases = ["做得好！", "太棒了！", "繼續保持這個節奏！", "你真棒！", "幹得漂亮！"]
    keep_going_phrases = ["明天再試一次！", "別氣餒！", "加油，別放棄！", "只差一點點了！"]

    # --- 2. 找出「已完成」的習慣 (包含備註) ---
    completed_df = df[df['Status'].astype(str) == '✅']
    completed_details = []

    # 逐行讀取，這樣才能把「備註」加進來
    for index, row in completed_df.iterrows():
        habit_name = row['Habit_Name']
        note = str(row.get('Notes', '')).strip() # .get() 確保 'Notes' 欄不存在也不會出錯

        if note: # 如果備註欄位不是空的
            completed_details.append(f"{habit_name} (備註: {note})")
        else:
            completed_details.append(habit_name)

    completed_count = len(completed_details)

    # --- 3. 找出「未完成」的習慣 ---
    missed_df = df[df['Status'].astype(str) == '❌']
    # 未完成的，我們只需要名稱列表 (並移除重複)
    missed_list = missed_df['Habit_Name'].unique()
    missed_count = len(missed_list)

    # --- 4. 組合「多行」的總結訊息 ---
    message_parts = [] # 我們用一個列表來組合訊息

    # (A) 處理「已完成」的部分
    if completed_count > 0:
        # 隨機選一個開頭
        message_parts.append(f"🌟 {random.choice(good_job_phrases)} 你今天完成了 {completed_count} 項習慣：")
        # 逐項列出
        for item in completed_details:
            message_parts.append(f"  ✅ {item}")
    else:
        message_parts.append("🤔 今天還沒有完成的習慣紀錄喔。")

    # (B) 處理「未完成」的部分
    if missed_count > 0:
        message_parts.append(f"\n💪 {random.choice(keep_going_phrases)} 還有 {missed_count} 項習慣在等你：")
        missed_txt = "、".join(missed_list)
        message_parts.append(f"  ❌ {missed_txt}")

    # (C) 加上結尾
    if completed_count > 0 and missed_count == 0:
        message_parts.append("\n🎉 完美的一天！所有計畫的習慣都達成了！")
    elif missed_count > 0:
        message_parts.append("\n別忘了休息，明天繼續！")

    # 用「換行符號」 \n 把它們全部組合起來
    return "\n".join(message_parts)

# --- 主程式 (這部分和之前一樣，不用動) ---
if 'df_log' in locals():
    # 為了讓總結更即時，我們只分析「今天」的紀錄
    tz = pytz.timezone(TIMEZONE)
    today_date_str = datetime.now(tz).strftime("%Y-%m-%d")

    # 篩選出 DataFrame 中 'Date' 欄位等於今天的紀錄
    today_df = df_log[df_log['Date'].astype(str) == today_date_str]

    if today_df.empty:
        # 假設今天的日期是 2025-11-10
        # (因為我們在 Cell 4 寫入了 11-10 的資料)
        # 為了測試，我們先手動指定日期
        test_date_str = "2025-11-10"
        print(f"👉 找不到 {today_date_str} (今天) 的紀錄。")
        print(f"   (正在改用 {test_date_str} 的範例資料進行測試...)")
        today_df = df_log[df_log['Date'].astype(str) == test_date_str]

        if today_df.empty:
             print(f"👉 連 {test_date_str} 的紀錄也找不到，將總結所有歷史紀錄。")
             built_in_message = make_built_in_summary(df_log)
        else:
             built_in_message = make_built_in_summary(today_df)
    else:
        print(f"👉 Zheng zai fen xi {today_date_str} (jin tian) de {len(today_df)} bi ji lu...")
        built_in_message = make_built_in_summary(today_df)

    print("\n--- 📝 內建總結（加強版）---")
    print(built_in_message)
    print("---------------------------------")

else:
    print("❌ 找不到 'df_log'，請先成功執行【挑戰 1】。")
    built_in_message = "Error: 資料讀取失敗"

--- 正在執行【挑戰 2：內建規則總結 (加強版)】---
👉 Zheng zai fen xi 2025-11-10 (jin tian) de 3 bi ji lu...

--- 📝 內建總結（加強版）---
🌟 你真棒！ 你今天完成了 2 項習慣：
  ✅ 學日文 (備註: 背 20 個單字)
  ✅ 學 Python (備註: 練習 for 迴圈)

💪 明天再試一次！ 還有 1 項習慣在等你：
  ❌ 運動

別忘了休息，明天繼續！
---------------------------------


In [21]:
# 這是 Colab 筆記本的第七個儲存格
print(f"--- 正在執行【挑戰 3：資料同步回寫】---")

if 'built_in_message' in locals() and "Error" not in built_in_message:
    try:
        # 1. 取得 Built_in_Summary 分頁，若不存在則自動建立
        try:
            summary_sheet = gsheets.worksheet(WORKSHEET_SUMMARY)
            print(f"正在存取 '{WORKSHEET_SUMMARY}' 分頁...")
        except gspread.exceptions.WorksheetNotFound:
            print(f"找不到分頁 '{WORKSHEET_SUMMARY}'，將自動建立新分頁...")
            summary_sheet = gsheets.add_worksheet(title=WORKSHEET_SUMMARY, rows="100", cols="10")
            # 建立標頭
            headers = ['Log_Date', 'Summary_Message']
            summary_sheet.update('A1:B1', [headers])

        # 2. 取得台北時區的「今天」
        tz = pytz.timezone(TIMEZONE)
        today_str = datetime.now(tz).strftime("%Y-%m-%d %H:%M:%S")

        # 3. 準備要「附加」的新資料
        new_row = [today_str, built_in_message]

        # 4. 附加寫入 (append_row)
        summary_sheet.append_row(new_row)

        print(f"✅ 成功將「內建總結」回寫至 '{WORKSHEET_SUMMARY}' 分頁！")

    except Exception as e:
        print(f"❌ 回寫 '{WORKSHEET_SUMMARY}' 失敗: {e}")
else:
    print("❌ 找不到 'built_in_message' 或總結失敗，跳過回寫步驟。")

--- 正在執行【挑戰 3：資料同步回寫】---
正在存取 'Built_in_Summary' 分頁...
✅ 成功將「內建總結」回寫至 'Built_in_Summary' 分頁！
